In [1]:
import json

file_path = '../data/annotated_github_topics_wikidata.json'

In [2]:
qids = {}
with open(file_path, 'rt') as inf:
    for line in inf:
        obj = json.loads(line)
        qids[obj['Wikidata ID']] = obj

In [3]:
from wikidata.client import Client
from collections import defaultdict, Counter

client = Client()

types = defaultdict(set)
for q in qids:
    entity = client.get(q, load=True)

    try:
        entity_types = entity.data['claims']['P31']
    except:
        continue

    for etype in entity_types:
        try:
            type_id = etype['mainsnak']['datavalue']['value']['id']
            type_title = client.get(type_id, load=True).label.texts['en']
            types[q].add(type_id)
        except:
            continue


In [4]:
file_path = '../data/reconciled-gittopic.json'

i = 0
duplicates = defaultdict(list)
count = Counter()
with open(file_path, 'rt') as inf:
    for line in inf:
        obj = json.loads(line)
        candidates = obj['candidates']
        for candidate in candidates:
            candidate_types = set(candidate['types'].keys())
            if candidate['id'] in qids and obj['frequency'] > 10:
                i += 1
                duplicates[candidate['id']].append(obj['term'])
                count[candidate['id']] += obj['frequency']

In [5]:
sum(count.values())

133050

In [6]:
duplicates

defaultdict(list,
            {'Q197536': ['deep-learning',
              'deeplearning',
              'deep-learning-tutorial',
              'hierarchical-data',
              'deep-q-learning',
              'deeplearning-ai',
              'hierarchical',
              'machine-learning-tutorials'],
             'Q2539': ['machine-learning',
              'ml',
              'machinelearning',
              'datascience',
              'statistical-learning',
              'scientific-machine-learning'],
             'Q189053': ['cli',
              'command-line',
              'commandline',
              'commands',
              'cli-command',
              'command-line-interface',
              'commandline-interface'],
             'Q2526135': ['security'],
             'Q8513': ['database',
              'bdd',
              'db',
              'databases',
              'base64',
              'stock-data'],
             'Q20514253': ['blockchain', 'blockchain-technology'